In [18]:
import logging
import sys

import gym
import torch

import numpy as np

import neat.population as pop
import neat.experiments.pole_balancing.config as c
from neat.visualize import draw_net
from neat.phenotype.feed_forward import FeedForwardNet

import time
import optuna

In [19]:
config = c.PoleBalanceConfig

def objective(trial):

    config.SCALE_ACTIVATION = trial.suggest_float('S_ACT', 1 , 10, log = True)

    config.FITNESS_THRESHOLD = 100000.0

    config.POPULATION_SIZE = trial.suggest_int('POP_SIZE', 5, 200)
    config.NUMBER_OF_GENERATIONS = 100
    config.SPECIATION_THRESHOLD = trial.suggest_float('SPEC_THRESH', 1.0, 5.0)

    config.CONNECTION_MUTATION_RATE = trial.suggest_float('CONN_MUT_RATE', 0.1, 0.9)
    config.CONNECTION_PERTURBATION_RATE = trial.suggest_float('CONN_PERT_RATE', 0.1, 0.9)
    config.ADD_NODE_MUTATION_RATE = trial.suggest_float('ADD_NODE_RATE', 0.1, 0.9)
    config.ADD_CONNECTION_MUTATION_RATE = trial.suggest_float('ADD_CONN_RATE', 0.1, 0.9)

    config.CROSSOVER_REENABLE_CONNECTION_GENE_RATE = trial.suggest_float('CROSSOVER', 0.1, 0.9)

    # Top percentage of species to be saved before mating
    config.PERCENTAGE_TO_SAVE = 0.80
    ttc = []
    for _ in range(5):
        neat = pop.Population(config)
        start = time.time()
        solution, generation = neat.run()
        end = time.time()

        ttc.append(end-start)
    

    return np.mean(ttc)



In [31]:
#optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "example_study"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(direction = "maximize",study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=1)

trial = study.best_trial

print('Time to Converge: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))



[I 2022-12-22 17:15:56,229] Using an existing study with name 'example_study' instead of creating a new one.


Using an existing study with name 'example_study' instead of creating a new one.
Using an existing study with name 'example_study' instead of creating a new one.
Using an existing study with name 'example_study' instead of creating a new one.
Using an existing study with name 'example_study' instead of creating a new one.


2022-12-22 17:15:56,959 [MainThread  ] [population] [INFO ]  Finished Generation 1
2022-12-22 17:15:56,960 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 193.0
2022-12-22 17:15:56,960 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-22 17:15:57,880 [MainThread  ] [population] [INFO ]  Finished Generation 2
2022-12-22 17:15:57,881 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 229.0
2022-12-22 17:15:57,881 [MainThread  ] [population] [INFO ]  Best Genome Length 7

[W 2022-12-22 17:16:35,340] Trial 3 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/qm/3gmtl0v9389290bhcns4s42m0000gq/T/ipykernel_76210/3547123208.py", line 26, in objective
    solution, generation = neat.run()
  File "/Users/aidan.horn/Githu

Trial 3 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/qm/3gmtl0v9389290bhcns4s42m0000gq/T/ipykernel_76210/3547123208.py", line 26, in objective
    solution, generation = neat.run()
  File "/Users/aidan.horn/GithubRepos/pytorch-neat/pytorch-neat/neat/population.py", line 37, in run
    genome.fitness = max(0, self.Config.fitness_fn(genome))
  File "/Users/aidan.horn/GithubRepos/pytorch-neat/pytorch-neat/neat/experiments/pole_balancing/config.py", line 55, in fitness_fn
    pred = round(float(phenotype(input)))
  File "/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/aidan.horn/GithubRepos/pyt

KeyboardInterrupt: 

In [32]:
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))


2022-12-22 17:17:01,050 [MainThread  ] [utils     ] [INFO ]  NumExpr defaulting to 8 threads.


In [33]:
df

,number,value,params_ADD_CONN_RATE,params_ADD_NODE_RATE,params_CONN_MUT_RATE,params_CONN_PERT_RATE,params_CROSSOVER,params_POP_SIZE,params_SPEC_THRESH,params_S_ACT,state
0,0,-34.068371,0.330072,0.510224,0.614841,0.775264,0.447270,133,4.249714,1.232995,COMPLETE
1,1,-302.094024,0.291047,0.795114,0.531207,0.284476,0.866950,10,2.732311,6.096019,COMPLETE
2,2,25.134528,0.624694,0.566369,0.454864,0.236677,0.693780,93,3.222719,4.473306,COMPLETE
3,3,NaN,0.749148,0.599445,0.528266,0.683981,0.637721,196,1.748734,8.031825,FAIL


In [6]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_slice(study) 
optuna.visualization.plot_contour(study, params=['S_ACT', 'POP_SIZE', 'SPEC_THRESH', 'CONN_MUT_RATE'], target_name='Time to Converge')

[W 2022-12-22 16:31:48,201] Param POP_SIZE unique value length is less than 2.
[W 2022-12-22 16:31:48,202] Param SPEC_THRESH unique value length is less than 2.
[W 2022-12-22 16:31:48,202] Param S_ACT unique value length is less than 2.
[W 2022-12-22 16:31:48,203] Param CONN_MUT_RATE unique value length is less than 2.
[W 2022-12-22 16:31:48,203] Param SPEC_THRESH unique value length is less than 2.
[W 2022-12-22 16:31:48,203] Param S_ACT unique value length is less than 2.
[W 2022-12-22 16:31:48,204] Param CONN_MUT_RATE unique value length is less than 2.
[W 2022-12-22 16:31:48,204] Param POP_SIZE unique value length is less than 2.
[W 2022-12-22 16:31:48,205] Param S_ACT unique value length is less than 2.
[W 2022-12-22 16:31:48,205] Param CONN_MUT_RATE unique value length is less than 2.
[W 2022-12-22 16:31:48,205] Param POP_SIZE unique value length is less than 2.
[W 2022-12-22 16:31:48,206] Param SPEC_THRESH unique value length is less than 2.
/Users/aidan.horn/opt/anaconda3/envs/

In [7]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'CONN_MUT_RATE'], target_name='Time to Converge')

[W 2022-12-22 16:31:54,933] Param CONN_MUT_RATE unique value length is less than 2.


In [19]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'SPEC_THRESH'], target_name='Time to Converge')